# Guia de uso da API do DeepInfra


### Habemus acesso aos modelos do Deep Infra!! 🎊

É necessário seguir algumas regras:

1. Cada grupo (frente) tem sua própria chave de API, então cada grupo só deve usar sua chave.

2. Bom senso: os créditos não são infinitos, é para todo o projeto usar, então faça um bom uso.

3. Faça uma simulação do valor do experimento que você quer rodar antes de executar. Assim, terá uma ideia de quanto vai custar. O script abaixo pode ajudar nisso.

4. As chaves devem ser enviadas em grupos privados ou mensagens diretas de cada pessoa. Peça acesso à chave ao pós-graduando que trabalha com você. 😉

# Installing Libraries

In [32]:
!pip install openai
!pip install pandas
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.1 MB/s eta 0:00:00


In [33]:
from openai import OpenAI
import pandas as pd
import tiktoken

# Make your first inference.

In [15]:
# Create an openAi Client

API_KEY="YOUR PRIVATE KEY GOES HERE"
BASE_URL="https://api.deepinfra.com/v1/openai"

openai = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
)

In [16]:
def get_completion(prompt, model):
    chat_completion = openai.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )
    return chat_completion.choices[0].message.content

### Choose your model and prompt


Choose among the available models: https://deepinfra.com/models/text-generation

In [17]:
get_completion("What is the capital of France?", "deepseek-ai/DeepSeek-R1-Turbo")

"<think>\nOkay, so I need to figure out the capital of France. Let me start by recalling what I know. France is a country in Europe, right? I remember that Paris is a major city there. But wait, is Paris actually the capital? Sometimes capitals can be tricky because some countries have changed their capitals over time or have multiple important cities.\n\nLet me think. I've heard people talk about Paris as the city of lights, and I know the Eiffel Tower is there. Also, major events like the Tour de France finish in Paris. But does that make it the capital? Maybe I should cross-check with some historical facts. France has had a long history; was the capital always Paris? For example, during World War II, the government moved to Vichy, but I think that was a temporary situation. Post-war, the capital moved back to Paris. \n\nAnother way to verify: think about government buildings. The French President's residence is the Élysée Palace in Paris. The National Assembly and Senate are also lo

#Estimating your experiment cost
1. Load a CSV file (called 'data.csv') with input and output columns.

2. Define the price per token for your model below (here you can see the model pricing: https://deepinfra.com/pricing).

3. Run the code below, and you will have the estimated cost.

In [43]:
#Cost for each token
INPUT_COST = 0.23 / 1000000 # 0.23 == Cost for 1M token
OUTPUT_COST = 0.40 / 1000000

In [44]:
df = pd.read_csv("data.csv")
df

,input,output
0,AI will replace many jobs.,"While AI may displace some jobs, it can also c..."
1,Generative AI improves creativity.,"AI assists creativity, but it may also lead to..."
2,AI is biased.,Bias in AI is not inherent; it reflects biases...
3,Self-driving cars are safer than human drivers.,AI still struggles with unpredictable situatio...
4,AI should be regulated.,Overregulation may stifle innovation and slow ...
5,AI is making education more personalized.,Over-reliance on AI in education might reduce ...
6,Generative AI threatens intellectual property.,AI-generated content does not necessarily viol...
7,AI will revolutionize healthcare.,"AI aids in healthcare, but ethical concerns an..."
8,AI-powered chatbots improve customer service.,Chatbots lack human empathy and can struggle w...
9,Generative AI can spread misinformation.,AI tools can also be used to detect and counte...


In [45]:
MODEL='gpt-3.5-turbo' #Model tokenizer (não achei tokenizadores pra os modelos do deepseek então usei esse que serve pra o GPT)

def count_tokens(text):
    """Counts the number of tokens in a given text using OpenAI's tokenizer."""
    encoding = tiktoken.encoding_for_model(MODEL)
    return len(encoding.encode(text)) if pd.notna(text) else 0


# Count tokens
df["input_tokens"] = df["input"].apply(lambda x: count_tokens(str(x)))
df["output_tokens"] = df["output"].apply(lambda x: count_tokens(str(x)))

# Calculate cost
df["input_cost"] = df["input_tokens"] * INPUT_COST
df["output_cost"] = df["output_tokens"] * OUTPUT_COST
df["total_cost"] = df["input_cost"] + df["output_cost"]

total_input_tokens = df["input_tokens"].sum()
total_output_tokens = df["output_tokens"].sum()
total_cost = df["total_cost"].sum()

print(f"Total Input Tokens: {total_input_tokens}")
print(f"Total Output Tokens: {total_output_tokens}")
print(f"Estimated Total Cost: {total_cost:.5f} $")

Total Input Tokens: 137
Total Output Tokens: 348
Estimated Total Cost: 0.00017 $
